In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/installation")

docs = loader.load()

In [2]:
import os
from langchain_openai import AzureOpenAIEmbeddings

os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = "<api-key>"

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="ada-embedding"
)

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

In [5]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

retriever = vector.as_retriever()

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [6]:
from langchain_core.messages import HumanMessage, AIMessage

context = [HumanMessage(content="Can I install langchain directly from source?"), AIMessage(content="Yes!")]

query = "Tell me how"

output = retrieval_chain.invoke({
    "context": context,
    "input": query
})

print(output["answer"])

To install LangChain, you have two options: the official release or from source.

1. Official release:
   - To install LangChain, run the following command:
     ```
     pip install langchain
     ```
     or
     ```
     conda install langchain -c conda-forge
     ```
   - This will install the basic requirements of LangChain.

2. From source:
   - If you prefer to install from source, you can clone the LangChain repository and navigate to the `langchain` directory.
   - Then run the following command:
     ```
     pip install -e .
     ```

It's important to note that while the basic requirements of LangChain will be installed, the dependencies needed for specific integrations (with model providers, datastores, etc.) are not included by default. You will need to install these dependencies separately.

Additionally, there are other packages available within the LangChain ecosystem that you may find useful:

- LangChain community: Contains third-party integrations. Install with:
  `